In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('C:\\Users\\genri\\OneDrive\\Desktop\\Cookpad\\data_analyst_interview_data.csv')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573854 entries, 0 to 573853
Data columns (total 10 columns):
id               573854 non-null int64
user_id          573854 non-null int64
recipe_id        573854 non-null int64
event_time       573854 non-null object
event            573854 non-null object
view_duration    221593 non-null float64
country          573854 non-null object
platform         573717 non-null object
ref              38545 non-null object
via              9703 non-null object
dtypes: float64(1), int64(3), object(6)
memory usage: 43.8+ MB


In [320]:
# let's check null 'view_duration' values for all events 
a = pd.DataFrame(data['event'].value_counts())
b = pd.DataFrame(data.view_duration.isnull().groupby([data['event']]).sum().astype(int))
funnel_final = pd.merge(left = a,right = b ,how = 'inner',left_index = True , right_index = True)
funnel_final['null_rate'] = 100 * funnel_final['view_duration'] / funnel_final['event']
funnel_final

,event,view_duration,null_rate
recipe.visit,325764,325764,100.000000
view_duration,228564,6971,3.049912
recipe.bookmark,10291,10291,100.000000
recipe.add_to_cookplan,2278,2278,100.000000
recipe.comments.visit,2158,2158,100.000000
recipe.react,1584,1584,100.000000
recipe.cooked_it,1066,1066,100.000000
recipe.share,673,673,100.000000
cookplan.comments.create,484,484,100.000000
user.follow,413,413,100.000000


In [214]:
#Get new feature - hour
data['event_time'] = pd.to_datetime(data['event_time'])
data['event_date'] = data['event_time'].dt.hour

In [221]:
#Function to convert hour to part of the day
def get_date_part(x):
    if x>=3 and x<=9:
        part = 'morning'
    elif x>=10 and x<=18:
        part = 'afternoon'
    elif x>18 and x<23:
        part = 'evening'
    else:
        part = 'night'
    return part

In [323]:
# Check null view duration across the platforms
# Interesting , there are no such values for IOS . Should be investigated it later
a = pd.DataFrame(data['platform'].value_counts())
b = pd.DataFrame(dt['platform'].value_counts())
funnel_final = pd.merge(left = a,right = b ,how = 'left',left_index = True , right_index = True)
funnel_final['null_rate'] = 100 * funnel_final['platform_y'] / funnel_final['platform_x']
funnel_final

,platform_x,platform_y,null_rate
app-android,378690,5197.0,1.372363
web-mobile,131688,1754.0,1.331936
app-ios,57281,NaN,NaN
web-desktop,5659,11.0,0.194381
web-tablet,399,9.0,2.255639


In [324]:
dt['platform'].value_counts()

app-android    5197
web-mobile     1754
web-desktop      11
web-tablet        9
Name: platform, dtype: int64

In [5]:
#Let's just get rid of event where view_duration is null
dt = data[(data['event'] == 'view_duration') & (data['view_duration'].isnull() == True )]
data_new = data.drop(dt.index)

In [ ]:
a = pd.DataFrame(data['ref'].value_counts())
b = pd.DataFrame(dt['platform'].value_counts())
funnel_final = pd.merge(left = a,right = b ,how = 'left',left_index = True , right_index = True)
funnel_final['null_rate'] = 100 * funnel_final['platform_y'] / funnel_final['platform_x']

In [84]:
data['via'].value_counts()

recipe_swipe                          3582
add_to_cookplan_button                1161
add_to_cookplan_steps_button           913
cooked_it_button                       795
question_card                          662
unknown                                379
recipe                                 371
comment_body                           225
cooked                                 223
author                                 219
post_button                            212
cooklog                                192
share_your_photo_button                165
open_log                               139
cookplan_tray_popup                     86
dialog                                  72
view_button                             62
profile                                 57
add_to_cookplan_toolbar                 42
recipe_card_swipe                       35
comment_to_author                       26
ask_question_button                     23
recipe_screenshot                       16
add_to_cook

In [50]:
data['ref'].value_counts()

Feed                                 18572
RecipeView                            4387
uncooked_recipes                      3024
feed                                  2265
profile                               2235
recipe_page                           1819
my_recipe_cookplan                     967
UserProfile                            897
unknown                                840
Search                                 776
cooked_recipes                         669
photo_comment_preview                  632
inbox                                  425
recipe_ideas                           336
cooking_logs                           225
recipe_editor                          212
my_recipe_uncooked                      60
feed_new_single                         57
my_recipe_cooked                        43
cookplan_tray                           35
feed_from_your_network_single           23
achievement                             10
search_result                           10
suggestions

In [51]:
data['via'].value_counts()

recipe_swipe                          3582
add_to_cookplan_button                1161
add_to_cookplan_steps_button           913
cooked_it_button                       795
question_card                          662
unknown                                379
recipe                                 371
comment_body                           225
cooked                                 223
author                                 219
post_button                            212
cooklog                                192
share_your_photo_button                165
open_log                               139
cookplan_tray_popup                     86
dialog                                  72
view_button                             62
profile                                 57
add_to_cookplan_toolbar                 42
recipe_card_swipe                       35
comment_to_author                       26
ask_question_button                     23
recipe_screenshot                       16
add_to_cook

In [23]:
#Create cohort data where cohort - min date for every customer
group = data_new.groupby(['user_id'])
cohort_data = group['event_date'].min()
cohort_data = cohort_data.reset_index()

In [40]:
#Calculate data to gauge retention 
a = data_new[['event_date','user_id']].drop_duplicates()
b = cohort_data
funnel_final = pd.merge(left = a,right = b ,how = 'left',left_on = 'user_id' , right_on = 'user_id')
funnel_final['event_date_y'] = funnel_final['event_date_y'].astype('datetime64')
funnel_final['event_date_x'] = funnel_final['event_date_x'].astype('datetime64')
funnel_final['diff'] = (funnel_final['event_date_x'] - funnel_final['event_date_y'])/np.timedelta64(1,'D')

In [101]:
#Retention Matrix
column_names = ['date','total_users','users_D1','users_D2','users_D3','users_D4','users_D5','users_D6','users_D7']
retention_matrix = pd.DataFrame(columns = column_names)
retention_matrix.date = [1]*31

In [116]:
#Calculate retention
j = 0
for date in pd.date_range(start="2020-07-01",end="2020-07-31"):
    users = funnel_final[funnel_final['event_date_y']==date]['user_id'].drop_duplicates()
    retention_matrix.iloc[j,0] = date
    retention_matrix.iloc[j,1] = len(users)
    for i in range(1,8):
        users_retent = funnel_final[(funnel_final['event_date_x'] == date + timedelta(days=i)) & (funnel_final['user_id'].isin(users))].drop_duplicates()
        retention_matrix.iloc[ j , i + 1 ] = len(users_retent)
    j += 1

In [119]:
#Save matrix
retention_matrix.to_csv('retention_cookPad.csv',index = False)

In [122]:
##Create cohort data where cohort - min date for every customer and for every via
group = data_new.groupby(['user_id','via'])
cohort_data = group['event_date'].min()
cohort_data = cohort_data.reset_index()
cohort_data['event_date'] =  cohort_data['event_date'].astype('datetime64')

In [ ]:
#Create VIA Matrix
column_names = ['screen','total_users','users_D1','users_D2','users_D3','users_D4','users_D5','users_D6','users_D7']
retention_matrix_screen = pd.DataFrame(columns = column_names)
retention_matrix_screen.screen = [1]*30

In [ ]:
#Create data to gauge via retention
a = data_new[['event_date','user_id','via']].drop_duplicates()
b = cohort_data
funnel_final = pd.merge(left = a,right = b ,how = 'left',left_on = 'user_id' , right_on = 'user_id')
cohort_data['event_date'] =  cohort_data['event_date'].astype('datetime64')
funnel_final['diff'] = (funnel_final['event_date_x'] - funnel_final['event_date_y'])/np.timedelta64(1,'D')

In [ ]:
#Retention VIA Matrix
j = 0
for via in cohort_data['via'].drop_duplicates():    
    users = cohort_data[['user_id','event_date']][cohort_data['via']==via].drop_duplicates()
    final = pd.merge(left = funnel_final,right = users ,how = 'left',left_on = 'user_id' , right_on = 'user_id')
    final.dropna(subset=['event_date'] , inplace = True)
    total = len(final['user_id'].drop_duplicates())
    retention_matrix_buttons.iloc[j,0] = via
    retention_matrix_buttons.iloc[j,1] = total
    for i in range(1,8):
        retention_matrix_buttons.iloc[ j , i + 1 ] = len(final['user_id'][final['event_date_x'] == final['event_date'] + timedelta(days=i)].drop_duplicates())
    j += 1

In [ ]:
#Save matrix Via
retention_matrix_buttons.to_csv('retention_buttons_cookPad.csv',index = False)

In [181]:
#create matrix screen
column_names = ['screen','total_users','users_D1','users_D2','users_D3','users_D4','users_D5','users_D6','users_D7']
retention_matrix_screen = pd.DataFrame(columns = column_names)
retention_matrix_screen.screen = [1]*30

In [186]:
##Create cohort data where cohort - min date for every customer and for every screen
group = data_new.groupby(['user_id','ref'])
cohort_data = group['event_date'].min()
cohort_data = cohort_data.reset_index()
cohort_data['event_date'] =  cohort_data['event_date'].astype('datetime64')

In [196]:
#Retention screen Matrix
j = 0
for ref in cohort_data['ref'].drop_duplicates():    
    users = cohort_data[['user_id','event_date']][cohort_data['ref']==ref].drop_duplicates()
    final = pd.merge(left = funnel_final,right = users ,how = 'left',left_on = 'user_id' , right_on = 'user_id')
    final.dropna(subset=['event_date'] , inplace = True)
    total = len(final['user_id'].drop_duplicates())
    retention_matrix_screen.iloc[j,0] = ref
    retention_matrix_screen.iloc[j,1] = total
    for i in range(1,8):
        retention_matrix_screen.iloc[ j , i + 1 ] = len(final['user_id'][final['event_date_x'] == final['event_date'] + timedelta(days=i)].drop_duplicates())
    j += 1

In [199]:
#Save it
retention_matrix_screen.to_csv('retention_screens_cookPad.csv',index = False)

In [245]:
#Create data to gauge AWR retention
group = data_new.groupby(['user_id'])
cohort_data = group['event_date'].min()
cohort_data = cohort_data.reset_index()

In [325]:
#create matrix AWR
column_names = ['date','user_enganged','users_churned','retention_days']
retention_mat_average_recipes = pd.DataFrame(columns = column_names)
retention_mat_average_recipes.date = [1]*31*7

In [315]:
#Calculate average viewed recipes for different retention periods
j = 0
cohort_data['event_date'] = cohort_data['event_date'].astype('datetime64')
data_new['event_date'] = data_new['event_date'].astype('datetime64')
for retent_days in range(1,8):
    for date in pd.date_range(start="2020-07-01",end="2020-07-31"):
        
        k = cohort_data[cohort_data['event_date']==date]
        k_eng = k[k['user_id'].isin(data_new[data_new['event_date']==date + timedelta(days=retent_days)]['user_id'])]
        
        av_eng = data_new[(data_new['user_id'].isin(k_eng['user_id'])) & 
         (data_new['event_date']>=pd.Timestamp(date)) & 
         (data_new['event_date']<=pd.Timestamp(date + timedelta(days=retent_days)))].groupby('user_id').recipe_id.nunique().mean()
        av_ret = data_new[(~data_new['user_id'].isin(k_eng['user_id'])) & 
         (data_new['event_date']>=pd.Timestamp(date)) & 
         (data_new['event_date']<=pd.Timestamp(date + timedelta(days=retent_days)))].groupby('user_id').recipe_id.nunique().mean()
        
        retention_mat_average_recipes.iloc[j,0] = date
        retention_mat_average_recipes.iloc[j,1] = av_eng
        retention_mat_average_recipes.iloc[j,2] = av_ret
        retention_mat_average_recipes.iloc[j,3] = retent_days
        j +=1

In [317]:
#save AWR
retention_mat_average_recipes.to_csv('average_recipes.csv',index = False)